# 03 - Z-Score Anomaly Detection

## Method 1 of 4: Statistical Approach

### What is Z-Score?

Z-Score measures **how far a value is from the average**, in terms of standard deviations.

**Formula:**
```
Z-Score = (value - mean) / standard_deviation
```

### The Rule

- 99.7% of data has Z-Score between -3 and +3
- If |Z-Score| > 3 → Point is in the 0.3% most extreme → **ANOMALY**

### Pros & Cons

| Pros | Cons |
|------|------|
| Simple & fast | Assumes normal distribution |
| Easy to interpret | Looks at features independently |
| No training needed | Misses multivariate anomalies |

---
## Step 1: Setup
---

In [1]:
# ═══════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, mean, stddev, when, lit, sqrt,
    abs as spark_abs,
    monotonically_increasing_id
)
import os

print("✓ Libraries imported")

✓ Libraries imported


In [2]:
# ═══════════════════════════════════════════════════════════════════
# CREATE SPARK SESSION
# ═══════════════════════════════════════════════════════════════════

spark = SparkSession.builder \
    .appName("ZScore_AnomalyDetection") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✓ Spark Session created")
print(f"  Version: {spark.version}")

✓ Spark Session created
  Version: 3.5.0


In [3]:
# ═══════════════════════════════════════════════════════════════════
# CONFIGURATION
# ═══════════════════════════════════════════════════════════════════

# File paths
DATA_PATH = "../data/processed/BTCUSDT_1h_processed.csv"
OUTPUT_PATH = "../data/results/"

# Feature columns (the 6 we engineered)
FEATURE_COLUMNS = [
    "return",
    "log_return",
    "volatility_24h",
    "volume_change",
    "volume_ratio",
    "price_range"
]

# Z-Score threshold (|z| > 3 is anomaly)
ZSCORE_THRESHOLD = 3.0

print("✓ Configuration set")
print(f"  Threshold: |Z-Score| > {ZSCORE_THRESHOLD}")

✓ Configuration set
  Threshold: |Z-Score| > 3.0


---
## Step 2: Load Data
---

In [5]:
# ═══════════════════════════════════════════════════════════════════
# LOAD DATA
# ═══════════════════════════════════════════════════════════════════

df = spark.read.csv(DATA_PATH, header=True, inferSchema=True)

print("✓ Data loaded")
print(f"  Rows: {df.count()}")
print(f"  Columns: {len(df.columns)}")

✓ Data loaded
  Rows: 976
  Columns: 12


In [6]:
# Quick look at the data
df.select(["timestamp"] + FEATURE_COLUMNS).show(5)

+-------------------+--------------------+-------------------+------------------+-------------------+-------------------+-------------------+
|          timestamp|              return|         log_return|    volatility_24h|      volume_change|       volume_ratio|        price_range|
+-------------------+--------------------+-------------------+------------------+-------------------+-------------------+-------------------+
|2025-12-18 05:00:00|-0.37403145469341315|-0.3747327014740492|0.8377261343202294| -6.068965551312633|0.45308684511986347| 0.4444308192071094|
|2025-12-18 06:00:00| 0.23719095937224655|0.23691010563498532|0.8384049004109412| -36.41353998942414| 0.2911791115236443|0.28736491142746723|
|2025-12-18 07:00:00|  0.2168711213556218|0.21663629539107865|0.8371344536961494| 116.84208166771897| 0.6346708263978863| 0.3117217895695883|
|2025-12-18 08:00:00| 0.16714681741587523|0.16700728258672656|0.8226760199490266|  33.33171089489788| 0.8360049578067108| 0.5071110284346882|
|2025-

---
## Step 3: Calculate Statistics
---

In [7]:
# ═══════════════════════════════════════════════════════════════════
# CALCULATE MEAN AND STDDEV FOR EACH FEATURE
# ═══════════════════════════════════════════════════════════════════

# Store statistics for each feature
stats = {}

for feature in FEATURE_COLUMNS:
    # agg() calculates aggregate statistics
    feature_stats = df.agg(
        mean(col(feature)).alias("mean"),
        stddev(col(feature)).alias("stddev")
    ).first()
    
    stats[feature] = {
        "mean": feature_stats["mean"],
        "stddev": feature_stats["stddev"]
    }

# Display statistics
print("Feature Statistics:")
print("=" * 60)
print(f"{'Feature':<20} {'Mean':>15} {'StdDev':>15}")
print("-" * 60)
for feature, values in stats.items():
    print(f"{feature:<20} {values['mean']:>15.4f} {values['stddev']:>15.4f}")

Feature Statistics:
Feature                         Mean          StdDev
------------------------------------------------------------
return                        0.0034          0.3741
log_return                    0.0027          0.3744
volatility_24h                0.3424          0.1689
volume_change                20.9250         93.3035
volume_ratio                  1.0466          0.9030
price_range                   0.4972          0.3956


---
## Step 4: Calculate Z-Scores
---

In [8]:
# ═══════════════════════════════════════════════════════════════════
# CALCULATE Z-SCORE FOR EACH FEATURE
# ═══════════════════════════════════════════════════════════════════

# Formula: z = (value - mean) / stddev

df_zscore = df

for feature in FEATURE_COLUMNS:
    feature_mean = stats[feature]["mean"]
    feature_std = stats[feature]["stddev"]
    
    # Add z-score column for this feature
    df_zscore = df_zscore.withColumn(
        f"{feature}_zscore",
        (col(feature) - lit(feature_mean)) / lit(feature_std)
    )

print("✓ Z-Scores calculated")
print("\nNew columns added:")
for feature in FEATURE_COLUMNS:
    print(f"  • {feature}_zscore")

✓ Z-Scores calculated

New columns added:
  • return_zscore
  • log_return_zscore
  • volatility_24h_zscore
  • volume_change_zscore
  • volume_ratio_zscore
  • price_range_zscore


In [9]:
# Show sample z-scores
df_zscore.select(
    "timestamp", "return", "return_zscore", "volume_change", "volume_change_zscore"
).show(5)

+-------------------+--------------------+-------------------+-------------------+--------------------+
|          timestamp|              return|      return_zscore|      volume_change|volume_change_zscore|
+-------------------+--------------------+-------------------+-------------------+--------------------+
|2025-12-18 05:00:00|-0.37403145469341315|-1.0088702903360094| -6.068965551312633| -0.2893138244392205|
|2025-12-18 06:00:00| 0.23719095937224655| 0.6250018267347827| -36.41353998942414| -0.6145383520084072|
|2025-12-18 07:00:00|  0.2168711213556218| 0.5706844195279328| 116.84208166771897|   1.028011859147836|
|2025-12-18 08:00:00| 0.16714681741587523|0.43776528587329255|  33.33171089489788| 0.13297143150529103|
|2025-12-18 09:00:00|  0.3471758348971177| 0.9190048234905341|-17.730478385481195|-0.41429860581981287|
+-------------------+--------------------+-------------------+-------------------+--------------------+
only showing top 5 rows



---
## Step 5: Flag Anomalies
---

In [10]:
# ═══════════════════════════════════════════════════════════════════
# FLAG ANOMALIES: |Z-Score| > THRESHOLD in ANY feature
# ═══════════════════════════════════════════════════════════════════

# Build condition: |z1| > 3 OR |z2| > 3 OR |z3| > 3 ...
anomaly_condition = None

for feature in FEATURE_COLUMNS:
    zscore_col = f"{feature}_zscore"
    condition = spark_abs(col(zscore_col)) > ZSCORE_THRESHOLD
    
    if anomaly_condition is None:
        anomaly_condition = condition
    else:
        anomaly_condition = anomaly_condition | condition  # OR

# Add anomaly label column
df_zscore = df_zscore.withColumn(
    "is_anomaly",
    when(anomaly_condition, 1).otherwise(0)
)

print("✓ Anomalies flagged")

✓ Anomalies flagged


In [11]:
# ═══════════════════════════════════════════════════════════════════
# COUNT RESULTS
# ═══════════════════════════════════════════════════════════════════

total = df_zscore.count()
anomalies = df_zscore.filter(col("is_anomaly") == 1).count()
normal = total - anomalies

print("\n" + "=" * 50)
print("Z-SCORE RESULTS")
print("=" * 50)
print(f"Total data points:  {total}")
print(f"Normal:             {normal} ({100*normal/total:.1f}%)")
print(f"Anomalies:          {anomalies} ({100*anomalies/total:.1f}%)")
print("=" * 50)


Z-SCORE RESULTS
Total data points:  976
Normal:             909 (93.1%)
Anomalies:          67 (6.9%)


---
## Step 6: Examine Anomalies
---

In [12]:
# ═══════════════════════════════════════════════════════════════════
# VIEW DETECTED ANOMALIES
# ═══════════════════════════════════════════════════════════════════

print("Sample anomalies detected:")
print("-" * 50)

df_zscore.filter(col("is_anomaly") == 1) \
    .select("timestamp", "close", "return", "return_zscore", 
            "volume_change", "volume_change_zscore") \
    .show(10, truncate=False)

Sample anomalies detected:
--------------------------------------------------
+-------------------+--------+-------------------+-------------------+------------------+--------------------+
|timestamp          |close   |return             |return_zscore      |volume_change     |volume_change_zscore|
+-------------------+--------+-------------------+-------------------+------------------+--------------------+
|2025-12-18 13:00:00|88851.7 |1.8241005986821657 |4.867004986816566  |611.2129031317518 |6.326537866904136   |
|2025-12-18 14:00:00|88345.2 |-0.5700509950850696|-1.532854447455867 |-3.870643419949804|-0.2657528324050261 |
|2025-12-18 17:00:00|86483.56|-1.7388597268616413|-4.657222849353952 |189.18505375930982|1.803363280517209   |
|2025-12-18 19:00:00|84483.79|-1.7287542165871916|-4.630209586937135 |176.64166711136397|1.6689268261515067  |
|2025-12-18 21:00:00|85630.95|1.2393794708489336 |3.3039740425255144 |-47.05556140593964|-0.7285965140243653 |
|2025-12-19 03:00:00|86891.39|1.45

---
## Step 7: Save Results
---

In [13]:
# ═══════════════════════════════════════════════════════════════════
# SAVE RESULTS TO CSV
# ═══════════════════════════════════════════════════════════════════

# Create results directory
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Select columns to save
result_columns = ["timestamp", "close", "return", "volume_change", 
                  "volatility_24h", "is_anomaly"]

# Add row ID for later joining
df_result = df_zscore.withColumn("row_id", monotonically_increasing_id())
df_result = df_result.select(["row_id"] + result_columns)

# Rename anomaly column to indicate method
df_result = df_result.withColumnRenamed("is_anomaly", "anomaly_zscore")

# Save
output_file = OUTPUT_PATH + "zscore_results"
df_result.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_file)

print(f"✓ Results saved to: {output_file}")

✓ Results saved to: ../data/results/zscore_results


In [14]:
# ═══════════════════════════════════════════════════════════════════
# STOP SPARK
# ═══════════════════════════════════════════════════════════════════

spark.stop()
print("✓ Spark stopped")
print("\n" + "=" * 50)
print("Z-SCORE COMPLETE - Proceed to 04_kmeans.ipynb")
print("=" * 50)

✓ Spark stopped

Z-SCORE COMPLETE - Proceed to 04_kmeans.ipynb
